In [107]:
import sympy as smp

from sympy import symbols, Eq, Function, dsolve, lambdify
import numpy as np


import plotly.graph_objects as go
from plotly.subplots import make_subplots


## 1D Heat Conduction 

Setup: 
  * Ambient environment with temperature $T_{amb}$ and conductivity $k_{amb}$. 
  * Wall with an exterior surface at $x = W_{start}$, and an interior surface at $x = W_{end}$ that has a temperature of $T_{int}$. The wall has a conductivity of $k_{wall}$. 
  * Steady-state 

Equation for steady state heat conduction[^1] with no heat generation 
$\begin{equation}
\frac{d^2T}{dx^2} = 0 
\end{equation}$

Interface boundary condition at $x = W_{start}$
$\begin{equation}
T(x = W_{start}) = T_{amb}
\end{equation}$

$\begin{equation}
-k_{amb} \frac{\partial T_{amb}(x = W_{start})}{\partial x} = -k_{wall} \frac{\partial T_{wall}(x = W_{start})}{\partial x}
\end{equation}$

Fixed temperature boundary condition at $x = W_{end}$:
$\begin{equation}
T(x = W_{end}) = T_{int}
\end{equation}$



[^1]: or heat transfer more generally?

### Constants 

In [108]:
T_amb = 25 # C 
T_int= 23 #C 

W_start = 0 #m 
W_end = 0.3 #m

k_amb = 1 # conductivity units...
k_wall = 1 # conductivity units 

### Fixed temperature at both sides

In [109]:
# x, W_start, W_end, T_amb, T_int = symbols('x W_start W_end T_amb T_int', real=True, positive=True)
# T = symbols('T', cls=smp.Function)
# implicitly stating that T is a function of x only...
T = Function("T")(x)
T

T(x)

In [110]:
eqs = [
    Eq(T.diff(x,2), 0 ),
]
eqs = eqs[0]
eqs

Eq(Derivative(T(x), (x, 2)), 0)

In [111]:
ics = {
T.subs(x, W_start): T_amb, 
T.subs(x, W_end): T_int
}
ics

{T(0): 25, T(0.3): 23}

In [112]:
# the PDE is in eqs (the first parameter), we are solving for T (the second parameter), and the initial/boundary conditions (the third parameter)
# NOTE: if eqs is an array, the second parameter has to be an array also
Tx = dsolve(eqs, T, ics=ics)
Tx

Eq(T(x), 25.0 - 6.66666666666667*x)

In [113]:
Tx_fun = lambdify([x], Tx.rhs)
x_num = np.linspace(W_start,W_end,100)
Tx_fun(x_num)[0:4]

array([25.        , 24.97979798, 24.95959596, 24.93939394])

In [114]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_num, y=Tx_fun(x_num), mode='markers',))
fig.update_layout(
    xaxis_title="x [m]",
    yaxis_title="Temperature [ºC]",
    title="Fixed temperature boundary conditions, both sides")

### Gradient at both sides

In [115]:
flux = -10

ics = {
    T.diff().subs(x, W_start): flux,
    T.subs(x, W_end): T_amb,
    # # T.subs(x, W_end): T_int,
    # T.diff().subs(x, W_end): flux 
}

ics

{Subs(Derivative(T(x), x), x, 0): -10, T(0.3): 25}

With gradient at both sides, still need at one fixed temperature. If have more than one, cannot solve the equation. Why?

In [116]:
Tx = dsolve(eqs, T, ics=ics)
Tx

Eq(T(x), 28.0 - 10.0*x)

In [117]:
Tx.rhs.diff()

-10.0000000000000

Can see that the derivative is everywhere the same, which is why the fluxes at both sides must be the same.. in reality, dont even need to specify the flux at the second side..

In [118]:
Tx.rhs.integrate()

-5.0*x**2 + 28.0*x

In [119]:
Tx_fun = lambdify([x], Tx.rhs)
x_num = np.linspace(W_start,W_end,100)
Tx_fun(x_num)[0:4]

array([28.        , 27.96969697, 27.93939394, 27.90909091])

In [120]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_num, y=Tx_fun(x_num), mode='markers',))
fig.update_layout(
    xaxis_title="x [m]",
    yaxis_title="Temperature [ºC]",
    title="Gradient boundary condition @ both sides, with initial fixed")

### Interface boundary condition

In [121]:
Ta = Function("T_amb")(x)
Tw = Function("T_wall")(x)

Tw

T_wall(x)

In [122]:
eqs = [
    Eq(Ta.diff(x,2), 0 ),
    Eq(Tw.diff(x,2), 0 ),
]
eqs[1]

Eq(Derivative(T_wall(x), (x, 2)), 0)

In [148]:
t_amb = 25

In [149]:
# "amb start"
amb_start = -1 * W_end

ics = {
# T_amb
Ta.subs(x, amb_start): t_amb ,
Ta.subs(x, W_start): t_amb ,

# # interface
# Ta.diff().subs(x, W_start): Tw.diff().subs(x, W_start),
# Tw.diff().subs(x, W_start): Ta.diff().subs(x, W_start), 

Tw.subs(x, W_start): Ta.subs(x, W_start),
# Ta.subs(x, W_start): Tw.subs(x, W_start),
Tw.subs(x, W_end): T_int
}
ics

{T_amb(-0.3): 25, T_amb(0): 25, T_wall(0): T_amb(0), T_wall(0.3): 23}

In [150]:
Tx = dsolve(eqs, [Ta, Tw], ics=ics)
Tx

[Eq(T_amb(x), 25.0),
 Eq(T_wall(x), -x*(3.33333333333333*T_amb(0) - 76.6666666666667) + T_amb(0))]

In [136]:
Tx_fun = lambdify([x], Tx[1].rhs)
# Tx_fun
x_num = np.linspace(W_start,W_end,100)
Tx_fun(x_num)[0:4]

NameError: name 'T_amb' is not defined

### sympy example

In [17]:
f, g = symbols("f g", cls=Function)
x = symbols("x")


In [27]:
eqs = [
    Eq(f(x).diff(x), g(x)),
    Eq(g(x).diff(x), f(x))
]

eqs[0]

Eq(Derivative(f(x), x), g(x))

In [28]:
eqs[1]

Eq(Derivative(g(x), x), f(x))

In [34]:
gen = dsolve(eqs, [f(x), g(x)])
gen, len(gen)

([Eq(f(x), -C1*exp(-x) + C2*exp(x)), Eq(g(x), C1*exp(-x) + C2*exp(x))], 2)

In [32]:
gen[0]

Eq(f(x), -C1*exp(-x) + C2*exp(x))

In [33]:
gen[1]

Eq(g(x), C1*exp(-x) + C2*exp(x))

In [36]:
spec = dsolve(eqs, [f(x), g(x)], ics={f(0): 1, g(2): 3})
spec, len(spec)

([Eq(f(x), (1 + 3*exp(2))*exp(x)/(1 + exp(4)) - (-exp(4) + 3*exp(2))*exp(-x)/(1 + exp(4))),
  Eq(g(x), (1 + 3*exp(2))*exp(x)/(1 + exp(4)) + (-exp(4) + 3*exp(2))*exp(-x)/(1 + exp(4)))],
 2)

In [37]:
spec[0]

Eq(f(x), (1 + 3*exp(2))*exp(x)/(1 + exp(4)) - (-exp(4) + 3*exp(2))*exp(-x)/(1 + exp(4)))